<a href="https://colab.research.google.com/github/Dulavinya/Tea_defect_detection/blob/main/Fertilizer_Deficiency_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import re
import requests
import csv
import tensorflow as tf
import math
import random

from sklearn.metrics import confusion_matrix,  multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from keras.utils import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout
from keras.callbacks import ModelCheckpoint , EarlyStopping
from keras.utils import Sequence

In [2]:
import sys
print("python version :",sys.version)
print("tensorflow version :",tf.__version__)

python version : 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
tensorflow version : 2.19.0


In [6]:
from typing import Tuple

class Dataset(Sequence):
    def __init__(self, paths: str, batch_size: int,input_shape:Tuple[int,int,int]):
        super(Dataset, self).__init__()
        self.batch_size = batch_size
        self.__data = paths
        self.classes = []
        self.image_size=input_shape
        self.__labels = self.__list_labels(self.__data)



    def __len__(self):
        return math.ceil(len(self.__data) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.__data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.__labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array([self.__load_image(i) for i in batch_x]), np.array(batch_y)

    def __list_labels(self, paths: list):
        labels = []
        for path in paths:
            #label = path.split(os.path.sep)[-2].split('_')
            label = path.split(os.path.sep)[-2].split(' ')
            labels.append(label)

        mlb = MultiLabelBinarizer()
        labels = mlb.fit_transform(labels)
        self.classes = mlb.classes_

        return labels

    @staticmethod
    def __load_image(path: str):
        img = cv2.imread(path)
        img = cv2.resize(img, (input_shape[0],input_shape[1]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img / 255.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')